In [1]:
import numpy as np



## Day1

https://adventofcode.com/2024/day/1

In [2]:
#Part 1
data = np.loadtxt("./puzzle_files/aoc1.txt")
print(np.abs(np.sort(data[:,0]) - np.sort(data[:,1])).sum())

2815556.0


In [3]:
# Part 2
ans = 0
for k in data[:,0]:
    ans += k*(data[:,1] == k).sum()
print(ans)

23927637.0


## Day 2

https://adventofcode.com/2024/day/2

In [4]:
with open("./puzzle_files/aoc2.txt", "r") as f:
    lines = f.readlines()
    #print(lines)

def report_safe(dd):
    return (((dd == np.sort(dd)).all() or (dd == np.sort(dd)[::-1]).all()) and (np.abs(np.diff(dd)) <= 3).all() and (np.abs(np.diff(dd)) >= 1).all())

In [5]:
# Part 1
ans = 0
for line in lines:
    dd = np.array(line.split(), dtype=np.float64)
    if report_safe(dd):
        ans += 1

print(ans)

230


In [6]:
# Part 2
ans = 0
for line in lines:
    dd = np.array(line.split(), dtype=np.float64)
    if report_safe(dd):
        ans += 1
    else:
        for i in range(len(dd)):
            dd2 = np.delete(np.copy(dd), i)
            if report_safe(dd2):
                ans += 1
                break

print(ans)

301
